# First tests over sequence clustering

### Installations and imports

In [ ]:
# 1. RUN THIS
!pip install kmedoids

In [1]:
# 2. RUN THIS
!git clone https://github.com/AdrianoLusso/Clustering_healthcare_sequences.git

Cloning into 'Clustering_healthcare_sequences'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 20 (delta 3), reused 17 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 12.88 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [31]:
# 3. NECESSARY IMPORTS
import numpy as np
import pandas as pd
import os
from kmedoids import pam as Pam

### Read dataset


In [15]:
for dirname, _, filenames in os.walk('Clustering_healthcare_sequences/test/dataset'):
    for filename in filenames:
        if filename != 'sars_combined_sequences.csv':
            continue
        print(os.path.join(dirname, filename))
        df=pd.read_csv(os.path.join(dirname, filename))

Clustering_healthcare_sequences/test/dataset/sars_combined_sequences.csv


### Plot dataset

In [18]:
df

,sequence,id,description,label
0,ATATTAGGTTTTTACCTTCCCAGGTAACAAACCAACTAACTCTCGA...,MG772934.1,MG772934.1 Bat SARS-like coronavirus isolate b...,bat-SARS
1,ATATTAGGTTTTTACCTTCCCAGGTAACAAACCAACTAACTCTCGA...,MG772933.1,MG772933.1 Bat SARS-like coronavirus isolate b...,bat-SARS
2,ATATTAGGTTTTTACCTACCCAGGAAAAGCCAACCAACCTCGATCT...,KY417152.1,KY417152.1 Bat SARS-like coronavirus isolate R...,bat-SARS
3,ATATTAGGTTTTTACCTACCCAGGAAAAGCCAACCAACCTCGATCT...,KY417151.1,KY417151.1 Bat SARS-like coronavirus isolate R...,bat-SARS
4,ATATTAGGTTTTTACCTACCCAGGAAAAGCCAACCAACCTCGATCT...,KY417150.1,KY417150.1 Bat SARS-like coronavirus isolate R...,bat-SARS
...,...,...,...,...
449,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253707.1,MT253707.1 Severe acute respiratory syndrome c...,SARS-cov-2
450,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253708.1,MT253708.1 Severe acute respiratory syndrome c...,SARS-cov-2
451,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253709.1,MT253709.1 Severe acute respiratory syndrome c...,SARS-cov-2
452,AGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...,MT253710.1,MT253710.1 Severe acute respiratory syndrome c...,SARS-cov-2


### Preprocess dataset

The dataset to be used will just be 100 sequences cut up up to 7 characters.

In [21]:
dataset = df.head(100)
sequences = [i[:7] for i in dataset['sequence'].tolist()]
print(sequences)

['ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'GTTAGGT', 'CCAGGAA', 'CCAGGAA', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'GTTAGGT', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'ATATTAG', 'AACTTTA', 'AACTTTA', 'AACTTTA', 'ATATTAG', 'ATATTAG', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CAGGAAA', 'CGATCTC', 'CGATCTC', 'CGATCTC', 'CGATCTC', 'CGATCTC', 'CGATCTC', 'CGATCTC', 'CGATCTC'

### Defining Optimal Matching algorithm

In [22]:
def optimal_matching(seq1, seq2, insert_cost=1, delete_cost=1, substitution_cost=None):
    global n_transitions
    global transitions_done

    n = len(seq1)
    m = len(seq2)

    # Initialize the distance matrix
    dist_matrix = np.zeros((n + 1, m + 1))

    # Fill the first row and column with cumulative insertion and deletion costs
    for i in range(1, n + 1):
        dist_matrix[i, 0] = dist_matrix[i - 1, 0] + delete_cost
    for j in range(1, m + 1):
        dist_matrix[0, j] = dist_matrix[0, j - 1] + insert_cost

    # Fill the distance matrix
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost_sub = substitution_cost(seq1[i - 1], seq2[j - 1]) if substitution_cost else (0 if seq1[i - 1] == seq2[j - 1] else 1)
            dist_matrix[i, j] = min(dist_matrix[i - 1, j] + delete_cost,  # Deletion
                                    dist_matrix[i, j - 1] + insert_cost,  # Insertion
                                    dist_matrix[i - 1, j - 1] + cost_sub)  # Substitution

    return dist_matrix[n, m]

In [23]:
n_transitions = 0
transitions_done = {}

# TODO don't use yet
def calculate_cost(a,b):

    if a == b:
        cost = 0
    else:
        n_transitions += 1
        if (a,b) in transitions_done.keys():
            transitions_done[(a,b)] += 1
        else:
            transitions_done[(a,b)] = 1

        p_ab = transitions_done[(a,b)] / n_transitions
        p_ba = transitions_done[(b,a)] / n_transitions if (b,a) in transitions_done.keys() else 0

        cost = 2 - p_ab - p_ba
    return cost

# This is the substitution cost calculation that will be used for now
def calculate_cost2(a, b):
    """Cálculo del costo de sustitución entre dos elementos."""
    return 0 if a == b else 1

In [24]:
# Example usage
sequence1 = ['A', 'B', 'C', 'A', 'B']
sequence2 = ['A', 'C', 'B', 'A']

#substitucion_cost = create_substitution_cost_trate_method()

distance = optimal_matching(sequence1, sequence2, substitution_cost=calculate_cost2)
print(f"Optimal Matching Distance: {distance}")

Optimal Matching Distance: 3.0


### Create the dissimilarities matrix

In [25]:
def generate_dissimilarities_matrix(sequences):
    n_sequences = len(sequences)
    matrix = np.zeros((n_sequences,n_sequences))

    for i in range(n_sequences):
        for j in range(n_sequences):
            if i == j:
                continue
            matrix[i,j] = optimal_matching(sequences[i],sequences[j], substitution_cost=calculate_cost2)

    return matrix


matrix=generate_dissimilarities_matrix(sequences)

In [26]:
matrix

array([[0., 0., 0., ..., 5., 5., 5.],
       [0., 0., 0., ..., 5., 5., 5.],
       [0., 0., 0., ..., 5., 5., 5.],
       ...,
       [5., 5., 5., ..., 0., 0., 0.],
       [5., 5., 5., ..., 0., 0., 0.],
       [5., 5., 5., ..., 0., 0., 0.]])

### Use the Partitioning Around Medoids algorithm

The algorithm been used and its result. We define a prefixed amount of clusters. The amount could be optimized using the silhoutte metric. (see Supporting Information: Appendix 4).

In [32]:
pam_result = Pam(matrix,4)

In [33]:
pam_result

KMedoidsResult(loss=16.0, labels=[1 1 1 1 1 1 1 1 1 1 1 1 1 3 0 0 1 1 1 1 1 1 1 3 3 3 3 3 3 3 3 3 3 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2], medoids=[47  0 83 13], n_iter=1, n_swaps=0)

There is kind of patterns found in the clusters. This will be the general workflow to implement with the real dataset.

In [41]:
for index,cluster in enumerate(pam_result.labels):
  print('label: ',dataset.iloc[index]['sequence'][:7],' , cluster: ',cluster)

label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  GTTAGGT  , cluster:  3
label:  CCAGGAA  , cluster:  0
label:  CCAGGAA  , cluster:  0
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  ATATTAG  , cluster:  1
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  GTTAGGT  , cluster:  3
label:  